# Movie Budget Data Analysis Project

## In this project we will be answering the following question: 
    How do the directors and producers, cast, production budget, and marketing of a movie impact the success of the movie pre-screening or post-screening?

## We have hypothesized the following: 
    We hypothesize that with better production, societally aware keywords during marketing, and a higher budget, movies will generally be more successful. However, we acknowledge that there are also movies that have a lower budget but more investment into the actors and playing to their genre that can stand as outliers and will need further consideration. 

## Background 
        From past experiences like Avatar big budget movies does not equate to the movie being successful at the box office. Movies can spend money on big names such as The Rock and special effects, but in the long run movies with a better plot have a better chance of making more money in the box office. Indie movies like Get Out that have low budgets and not a list actors have done better than bigger movies released on the same weekend. To confirm such we’ve taken a look at a couple data science projects that have looked into the profitability of movies as in: https://www.kaggle.com/param1/the-money-makers


![alt text](img.jpg "Title")

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt